In [ ]:
!pip install accelerate datasets flash-attn tiktoken transformers torch

In [ ]:
from datasets import load_dataset
data = load_dataset("facebook/flores","all")

In [ ]:
train_data = data['dev']